In [21]:
import pandas as pd

In [22]:

rna = pd.read_csv("rna_tpm.tsv", sep="\t", index_col=0) 
mirna = pd.read_csv("mirna.tsv", sep="\t", index_col=0).T
protein = pd.read_csv("protein.tsv", sep="\t", index_col=0).T

clinical = pd.read_csv("clinical.tsv", sep="\t", index_col=0)
subtypes = pd.read_csv("subtypes.csv", index_col=0)


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
print(rna)

                    TCGA-D8-A146-01A  TCGA-AQ-A0Y5-01A  TCGA-C8-A274-01A  \
Ensembl_ID                                                                 
ENSG00000000003.15          5.662037          3.703721          6.514515   
ENSG00000000005.6           3.376096          0.463099          0.000000   
ENSG00000000419.13          6.860140          7.086452          6.805072   
ENSG00000000457.14          4.400552          4.051007          5.037264   
ENSG00000000460.17          2.845169          2.426989          4.043248   
...                              ...               ...               ...   
ENSG00000288669.1           0.000000          0.000000          0.000000   
ENSG00000288670.1           3.533999          4.773448          4.701710   
ENSG00000288671.1           0.000000          0.000000          0.000000   
ENSG00000288674.1           0.058109          0.054779          0.093425   
ENSG00000288675.1           1.122739          1.243182          0.604261   

           

## Wybrór konkretnego RNA --- zamienic nazwy na geny !!!!!!

In [35]:
import pandas as pd

df = rna
# przykładowo:
# df = pd.read_csv("rna.tsv", sep="\t", index_col=0)

# usuń część po kropce z ID
df.index = df.index.str.replace(r'\.\d+$', '', regex=True)


In [36]:
import mygene
mg = mygene.MyGeneInfo()

mapping = mg.querymany(df.index, scopes='ensembl.gene', fields='symbol', species='human')


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
29 input query terms found dup hits:	[('ENSG00000175711', 2), ('ENSG00000188660', 2), ('ENSG00000215156', 2), ('ENSG00000226506', 2), ('E
1303 input query terms found no hit:	['ENSG00000002079', 'ENSG00000002586.20_PAR_Y', 'ENSG00000112096', 'ENSG00000124333.16_PAR_Y', 'ENSG


In [37]:
# słownik: Ensembl → symbol
map_dict = {d['query']: d.get('symbol') for d in mapping if 'symbol' in d}

# teraz podmień indeks w dataframe
df = df.rename(index=map_dict)


In [38]:
df = df[~df.index.isna()]


In [39]:
df = df[~df.index.str.startswith('ENSG')]


In [40]:
df.to_csv("rna_mapped.csv")


### wybór genów

In [41]:
rna_wybrane =  pd.read_csv("rna_mapped.csv")


In [42]:
genes = [
    "ESR1", "PGR", "ERBB2", "EGFR", "KRT5", "KRT14", "KRT17", "FOXA1", "GATA3",
    "BCL2", "CDH1", "VIM", "CLDN3", "CLDN4", "CLDN7", "MKI67", "CCND1", "MYC",
    "TP53", "PIK3CA", "PTEN", "AKT1", "BRCA1", "BRCA2", "AR", "FGFR1",
    "KRT18", "KRT19", "MUC1", "EPCAM", "SNAI1", "SNAI2", "ZEB1", "ZEB2",
    "TWIST1", "TGFBR2", "CD24", "CD44"
]

print(rna_wybrane)


         Ensembl_ID  TCGA-D8-A146-01A  TCGA-AQ-A0Y5-01A  TCGA-C8-A274-01A  \
0            TSPAN6          5.662037          3.703721          6.514515   
1              TNMD          3.376096          0.463099          0.000000   
2              DPM1          6.860140          7.086452          6.805072   
3             SCYL3          4.400552          4.051007          5.037264   
4             FIRRM          2.845169          2.426989          4.043248   
...             ...               ...               ...               ...   
45040  LOC128031834          0.000000          0.000000          0.000000   
45041        RNF228          0.151989          0.972619          0.389677   
45042  LOC101927145          0.000000          0.000000          0.000000   
45043  LOC128031833          0.000000          0.000000          0.000000   
45044         PANO1          1.122739          1.243182          0.604261   

       TCGA-BH-A0BD-01A  TCGA-B6-A1KC-01B  TCGA-AC-A62V-01A  TCGA-AO-A0J5-0

In [43]:
available_rna = [p for p in genes if p in rna_wybrane['Ensembl_ID'].values]
missing_rna = [p for p in genes if p not in rna_wybrane['Ensembl_ID'].values]

print("Dostępne geny:", available_rna)
print("Brakujące geny:", missing_rna)

# FILTRUJ WIERSZE GDZIE Ensembl_ID JEST W available_rna
selected_dfmirna = rna_wybrane[rna_wybrane['Ensembl_ID'].isin(available_rna)]
selected_dfmirna.to_csv("wybrane_rna.tsv", sep="\t", index=False)


Dostępne geny: ['ESR1', 'PGR', 'ERBB2', 'EGFR', 'KRT5', 'KRT14', 'KRT17', 'FOXA1', 'GATA3', 'BCL2', 'CDH1', 'VIM', 'CLDN3', 'CLDN4', 'CLDN7', 'MKI67', 'CCND1', 'MYC', 'TP53', 'PIK3CA', 'PTEN', 'AKT1', 'BRCA1', 'BRCA2', 'AR', 'FGFR1', 'KRT18', 'KRT19', 'MUC1', 'EPCAM', 'SNAI1', 'SNAI2', 'ZEB1', 'ZEB2', 'TWIST1', 'TGFBR2', 'CD24', 'CD44']
Brakujące geny: []


In [44]:
g = mg.getgene('ENSG00000000003',  field = 'symbol')# , 'symbol' ] )# fields= 'entrezgene, symbol') 

print(g)

{'_id': '7105', '_version': 1, 'symbol': 'TSPAN6'}


## Wybrór konkretnego miRNA

In [24]:
wybrane_mirna = [ 
    #bazal
    "hsa-mir-221",
    "hsa-mir-222",  
    "hsa-mir-17",
    "hsa-mir-150",
    "hsa-mir-210",
    "hsa-mir-191",
    "hsa-mir-133b",
    "hsa-mir-146b",
    "hsa-let-7f-1",
    "hsa-let-7f-2",

    # lumA
    "hsa-mir-23b",
    "hsa-mir-142",
    "hsa-mir-29a",
    "hsa-mir-181d",
    "hsa-mir-16",
    "hsa-mir-29b",
    "hsa-mir-155",
    "hsa-mir-181c",

    # lumB
    "hsa-mir-148",
    "hsa-let-7d",
    "hsa-mir-92a",
    "hsa-mir-34c",
    "hsa-let-7b",
    "hsa-mir-15a",

    # her2
    "hsa-mir-125b",
    "hsa-mir-134",
    "hsa-mir-98",
    "hsa-mir-143",
    "hsa-mir-138",
    "hsa-mir-135b"
]



availablemirna = [p for p in wybrane_mirna if p in mirna.columns]
missingmirna = [p for p in wybrane_mirna if p not in mirna.columns]

print(availablemirna)
print(missingmirna)


selected_dfmirna = mirna[availablemirna]
#print(selected_dfmirna)
selected_dfmirna.to_csv("wybrane_mirna.tsv", sep="\t")


['hsa-mir-221', 'hsa-mir-222', 'hsa-mir-17', 'hsa-mir-150', 'hsa-mir-210', 'hsa-mir-191', 'hsa-mir-133b', 'hsa-mir-146b', 'hsa-let-7f-1', 'hsa-let-7f-2', 'hsa-mir-23b', 'hsa-mir-142', 'hsa-mir-29a', 'hsa-mir-181d', 'hsa-mir-155', 'hsa-mir-181c', 'hsa-let-7d', 'hsa-mir-34c', 'hsa-let-7b', 'hsa-mir-15a', 'hsa-mir-134', 'hsa-mir-98', 'hsa-mir-143', 'hsa-mir-135b']
['hsa-mir-16', 'hsa-mir-29b', 'hsa-mir-148', 'hsa-mir-92a', 'hsa-mir-125b', 'hsa-mir-138']


## Wybrór konkretnego protein

In [25]:

proteins = [
    "ERALPHA", "PR", "HER2", "PHER2", "EGFR", "AKT_pS473", "PAKT", "P53", "PS6",
    "PTEN", "PMAPK", "MEK1", "CyclinB1", "Ki67", "BCL2", "BAX",
    "PARPCLEAVED", "p53", "BRCA1", "ECADHERIN", "NCADHERIN", "BETACATENIN", "PDL1"
]

protein.columns = protein.columns.str.replace(r"[^A-Za-z0-9]", "", regex=True).str.upper()

available = [p for p in proteins if p in protein.columns]
missing = [p for p in proteins if p not in protein.columns]


selected_df = protein[available]

selected_df.to_csv("protein_selected.tsv", sep="\t")

#print(selected_df.head())


In [18]:
#print("RNA IDs:", rna.index[:5])
#print("MiRNA IDs:", mirna.index[:5])
#print("Protein IDs:", protein.index[:5])
#print("Clinical IDs:", clinical.index[:5])
#print("Subtypes IDs:", subtypes.index[:5])


In [19]:
#common_ids = set(rna.index) & set(mirna.index) & set(protein.index)
#print("Liczba wspólnych pacjentek molekularnych:", len(common_ids))

#combined_molecular = rna.join([mirna, protein], how="inner")
#print("Rozmiar połączonych danych molekularnych:", combined_molecular.shape)

#print(combined_molecular.head())


# Wybór ze zbioru clinical

In [27]:
clinical = pd.read_csv("clinical-oczyszcz.tsv", sep="\t", index_col=0)


In [ ]:
#clinical.columns

In [28]:
selected_clinical = [
    "cases.submitter_id", "cases.case_id",
    "demographic.age_at_index", "demographic.gender",
    "demographic.race", "demographic.ethnicity",
    "demographic.vital_status", "demographic.days_to_death",
    "cases.days_to_lost_to_followup",
    "diagnoses.age_at_diagnosis", "diagnoses.primary_diagnosis",
    "diagnoses.ajcc_pathologic_stage", "diagnoses.ajcc_pathologic_t",
    "diagnoses.ajcc_pathologic_n", "diagnoses.ajcc_pathologic_m",
    "diagnoses.year_of_diagnosis", "diagnoses.laterality",
    "treatments.treatment_type", "treatments.treatment_outcome"
]


In [29]:
clinical_av = [p for p in selected_clinical if p in clinical.columns]
miss_clinic = [p for p in selected_clinical if p not in clinical.columns]

print(clinical_av)
print(miss_clinic)


selected_clinical= clinical[clinical_av]
selected_clinical.to_csv("clinical_av.tsv", sep="\t")

['cases.submitter_id', 'cases.case_id', 'demographic.age_at_index', 'demographic.gender', 'demographic.race', 'demographic.ethnicity', 'demographic.vital_status', 'demographic.days_to_death', 'cases.days_to_lost_to_followup', 'diagnoses.age_at_diagnosis', 'diagnoses.primary_diagnosis', 'diagnoses.ajcc_pathologic_stage', 'diagnoses.ajcc_pathologic_t', 'diagnoses.ajcc_pathologic_n', 'diagnoses.ajcc_pathologic_m', 'diagnoses.year_of_diagnosis', 'diagnoses.laterality', 'treatments.treatment_type', 'treatments.treatment_outcome']
[]


# Wybór ze zbioru subtypes


In [30]:
subtypes = pd.read_csv("subtypes.csv")


In [31]:
selected_subtypes = [
    "patient",
    "Tumor.Type",
    "vital_status",
    "days_to_death",
    "days_to_last_followup",
    "age_at_initial_pathologic_diagnosis",
    "pathologic_stage",
    "Tumor_Grade",
    "BRCA_Pathology",
    "BRCA_Subtype_PAM50",
    "tobacco_smoking_history"
]


In [32]:
subtypes_av = [p for p in selected_subtypes if p in subtypes.columns]
miss_subtypes= [p for p in selected_subtypes if p not in subtypes.columns]

print(subtypes_av)
print(miss_subtypes)


selected_subtypes2= subtypes[subtypes_av]
selected_subtypes2.to_csv("subtypes_av.tsv", sep="\t")

['patient', 'Tumor.Type', 'vital_status', 'days_to_death', 'days_to_last_followup', 'age_at_initial_pathologic_diagnosis', 'pathologic_stage', 'Tumor_Grade', 'BRCA_Pathology', 'BRCA_Subtype_PAM50', 'tobacco_smoking_history']
[]


# Porównanie ID pacjentów między plikami

## miRNA + protein

In [33]:

df_protein = pd.read_csv('protein_selected.tsv', sep='\t', index_col=0)
df_mirna = pd.read_csv('wybrane_mirna.tsv', sep='\t', index_col=0)

ids_protein = set(df_protein.index)
ids_mirna = set(df_mirna.index)


common_ids = len(ids_protein.intersection(ids_mirna))
different_ids = len(ids_protein.symmetric_difference(ids_mirna))

print(f"Liczba takich samych ID (wspólnych): {common_ids}")
print(f"Liczba różnych ID (łącznie w obu plikach): {different_ids}")


Liczba takich samych ID (wspólnych): 897
Liczba różnych ID (łącznie w obu plikach): 327


### Połączenie plików po id pacjenta

In [34]:

df_combined = df_protein.join(df_mirna, how='inner')

print(df_combined)

df_combined.to_csv("protein_mirna-connected.tsv", sep="\t")

                  ERALPHA        PR      HER2     EGFR      P53      PTEN  \
TCGA-WT-AB41-01A  3.85540 -0.219920 -0.093352 -0.57223 -0.19534  0.253260   
TCGA-BH-A0H5-01A  2.31530 -0.157000 -0.067206 -0.46202 -0.58071  0.173550   
TCGA-C8-A27A-01A  2.45540  0.071407  0.002673 -0.46112  0.36894 -0.565690   
TCGA-AC-A23C-01A  3.05480 -0.239620  2.002200 -0.22857 -1.15750 -0.007521   
TCGA-A8-A085-01A  3.27370 -0.178650  0.135140 -0.55378 -0.28867  0.664500   
...                   ...       ...       ...      ...      ...       ...   
TCGA-A2-A0CS-01A  2.67140  0.582550 -0.056856 -0.64176 -0.15178  0.150850   
TCGA-LL-A8F5-01A -1.09020 -0.171540 -0.346880  0.57231  0.54341 -0.459190   
TCGA-E9-A229-01A  0.79379  0.264860 -0.618610 -0.18988 -0.23675  0.116410   
TCGA-D8-A1Y0-01A  2.87270  0.359090 -0.282480 -0.16115  0.23611  1.006500   
TCGA-BH-A0DH-01A  2.41810  0.243550  0.233270 -0.29751 -0.44530  0.457910   

                     MEK1     BCL2       BAX  PARPCLEAVED  ...  hsa-mir-155

## miRNA + protein + RNA

In [45]:
rna_wybrane = pd.read_csv("wybrane_rna.tsv", sep="\t", index_col=0).T 
mirnaprot = pd.read_csv("protein_mirna-connected.tsv", sep="\t", index_col=0) 

In [46]:
print(mirnaprot)


                  ERALPHA        PR      HER2     EGFR      P53      PTEN  \
TCGA-WT-AB41-01A  3.85540 -0.219920 -0.093352 -0.57223 -0.19534  0.253260   
TCGA-BH-A0H5-01A  2.31530 -0.157000 -0.067206 -0.46202 -0.58071  0.173550   
TCGA-C8-A27A-01A  2.45540  0.071407  0.002673 -0.46112  0.36894 -0.565690   
TCGA-AC-A23C-01A  3.05480 -0.239620  2.002200 -0.22857 -1.15750 -0.007521   
TCGA-A8-A085-01A  3.27370 -0.178650  0.135140 -0.55378 -0.28867  0.664500   
...                   ...       ...       ...      ...      ...       ...   
TCGA-A2-A0CS-01A  2.67140  0.582550 -0.056856 -0.64176 -0.15178  0.150850   
TCGA-LL-A8F5-01A -1.09020 -0.171540 -0.346880  0.57231  0.54341 -0.459190   
TCGA-E9-A229-01A  0.79379  0.264860 -0.618610 -0.18988 -0.23675  0.116410   
TCGA-D8-A1Y0-01A  2.87270  0.359090 -0.282480 -0.16115  0.23611  1.006500   
TCGA-BH-A0DH-01A  2.41810  0.243550  0.233270 -0.29751 -0.44530  0.457910   

                     MEK1     BCL2       BAX  PARPCLEAVED  ...  hsa-mir-155

In [47]:
print(rna_wybrane)

Ensembl_ID           BRCA1     SNAI2       VIM      CD44      CDH1     FGFR1  \
TCGA-D8-A146-01A  2.302465  5.139527  8.685356  7.548715  8.436202  5.044901   
TCGA-AQ-A0Y5-01A  2.356848  4.864662  9.199860  7.579948  9.267621  5.104970   
TCGA-C8-A274-01A  3.380064  3.051407  6.699311  6.898577  4.394610  3.777798   
TCGA-BH-A0BD-01A  3.907535  4.749202  8.122898  7.890648  8.542904  4.896039   
TCGA-B6-A1KC-01B  3.630336  2.766362  6.600512  7.911637  8.792413  7.841624   
...                    ...       ...       ...       ...       ...       ...   
TCGA-BH-A0EI-01A  3.430580  5.199209  9.040569  8.633029  8.643919  8.136523   
TCGA-E2-A1IO-01A  2.873282  5.920365  8.401256  7.375676  8.800233  5.670237   
TCGA-E2-A15R-01A  4.589841  4.531849  7.050715  7.205621  8.334645  8.341313   
TCGA-B6-A0IP-01A  3.355341  4.076149  7.765050  8.370151  4.636903  3.872967   
TCGA-A1-A0SN-01A  3.504201  5.344083  8.211230  6.057288  8.443455  4.894590   

Ensembl_ID             PGR      ESR1   

In [48]:
df_combined2 = mirnaprot.join(rna_wybrane, how='inner', lsuffix='_mirnaprot', rsuffix='_rna')

df_combined2.to_csv("protein_mirna_rna-connected.tsv", sep="\t")

# clinical + subtypes

In [49]:
df_clinical = pd.read_csv('clinical_av.tsv', sep='\t', index_col=0)
df_subtypes = pd.read_csv('subtypes_av.tsv', sep='\t', index_col=0)

In [50]:
print(df_clinical)

                   cases.submitter_id                         cases.case_id  \
project.project_id                                                            
TCGA-BRCA                TCGA-E2-A1IU  001cef41-ff86-4d3f-a140-a647ac4b10a1   
TCGA-BRCA                TCGA-E2-A1IU  001cef41-ff86-4d3f-a140-a647ac4b10a1   
TCGA-BRCA                TCGA-E2-A1IU  001cef41-ff86-4d3f-a140-a647ac4b10a1   
TCGA-BRCA                TCGA-A1-A0SB  0045349c-69d9-4306-a403-c9c1fa836644   
TCGA-BRCA                TCGA-A2-A04W  00807dae-9f4a-4fd1-aac2-82eb11bf2afb   
...                               ...                                   ...   
TCGA-BRCA                TCGA-E9-A295  fec0da58-1047-44d2-b6d1-c18cceed43dc   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a-4c0f-8c0a-4259cd496e6d   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a-4c0f-8c0a-4259cd496e6d   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a-4c0f-8c0a-4259cd496e6d   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a

In [51]:
print(df_subtypes)

           patient Tumor.Type vital_status  days_to_death  \
0     TCGA-3C-AAAU       BRCA        Alive            NaN   
1     TCGA-3C-AALI       BRCA        Alive            NaN   
2     TCGA-3C-AALJ       BRCA        Alive            NaN   
3     TCGA-3C-AALK       BRCA        Alive            NaN   
4     TCGA-4H-AAAK       BRCA        Alive            NaN   
...            ...        ...          ...            ...   
1082  TCGA-WT-AB44       BRCA        Alive            NaN   
1083  TCGA-XX-A899       BRCA        Alive            NaN   
1084  TCGA-XX-A89A       BRCA        Alive            NaN   
1085  TCGA-Z7-A8R5       BRCA        Alive            NaN   
1086  TCGA-Z7-A8R6       BRCA        Alive            NaN   

      days_to_last_followup  age_at_initial_pathologic_diagnosis  \
0                    4047.0                                   55   
1                    4005.0                                   50   
2                    1474.0                                   6

In [52]:
merged_df = df_clinical.reset_index().merge(
    df_subtypes,
    how='inner',
    left_on='cases.submitter_id',
    right_on='patient'
)

#print("Liczba wspólnych pacjentów:", merged_df['patient'].nunique())

In [53]:
merged_df.to_csv('merged_clinical_subtypes.tsv', sep='\t', index=False)


# Clinical + subtypes + protein + miRNA + RNA

In [56]:
df_geny = pd.read_csv('protein_mirna_rna-connected.tsv', sep='\t', index_col=0)
df_ogolne = pd.read_csv('merged_clinical_subtypes.tsv', sep='\t', index_col=0)

In [57]:
df_geny.index = df_geny.index.str.replace(r'-\d+[A-Z]+$', '', regex=True)

In [62]:
print(df_ogolne)

                   cases.submitter_id                         cases.case_id  \
project.project_id                                                            
TCGA-BRCA                TCGA-E2-A1IU  001cef41-ff86-4d3f-a140-a647ac4b10a1   
TCGA-BRCA                TCGA-E2-A1IU  001cef41-ff86-4d3f-a140-a647ac4b10a1   
TCGA-BRCA                TCGA-E2-A1IU  001cef41-ff86-4d3f-a140-a647ac4b10a1   
TCGA-BRCA                TCGA-A1-A0SB  0045349c-69d9-4306-a403-c9c1fa836644   
TCGA-BRCA                TCGA-A2-A04W  00807dae-9f4a-4fd1-aac2-82eb11bf2afb   
...                               ...                                   ...   
TCGA-BRCA                TCGA-E9-A295  fec0da58-1047-44d2-b6d1-c18cceed43dc   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a-4c0f-8c0a-4259cd496e6d   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a-4c0f-8c0a-4259cd496e6d   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a-4c0f-8c0a-4259cd496e6d   
TCGA-BRCA                TCGA-C8-A26W  ff0f875f-113a

In [63]:
df_ogolne.columns

Index(['cases.submitter_id', 'cases.case_id', 'demographic.age_at_index',
       'demographic.gender', 'demographic.race', 'demographic.ethnicity',
       'demographic.vital_status', 'demographic.days_to_death',
       'cases.days_to_lost_to_followup', 'diagnoses.age_at_diagnosis',
       'diagnoses.primary_diagnosis', 'diagnoses.ajcc_pathologic_stage',
       'diagnoses.ajcc_pathologic_t', 'diagnoses.ajcc_pathologic_n',
       'diagnoses.ajcc_pathologic_m', 'diagnoses.year_of_diagnosis',
       'diagnoses.laterality', 'treatments.treatment_type',
       'treatments.treatment_outcome', 'patient', 'Tumor.Type', 'vital_status',
       'days_to_death', 'days_to_last_followup',
       'age_at_initial_pathologic_diagnosis', 'pathologic_stage',
       'Tumor_Grade', 'BRCA_Pathology', 'BRCA_Subtype_PAM50',
       'tobacco_smoking_history'],
      dtype='object')

In [65]:
df_geny.columns


Index(['ERALPHA', 'PR', 'HER2', 'EGFR_mirnaprot', 'P53', 'PTEN_mirnaprot',
       'MEK1', 'BCL2_mirnaprot', 'BAX', 'PARPCLEAVED', 'ECADHERIN',
       'NCADHERIN', 'BETACATENIN', 'PDL1', 'hsa-mir-221', 'hsa-mir-222',
       'hsa-mir-17', 'hsa-mir-150', 'hsa-mir-210', 'hsa-mir-191',
       'hsa-mir-133b', 'hsa-mir-146b', 'hsa-let-7f-1', 'hsa-let-7f-2',
       'hsa-mir-23b', 'hsa-mir-142', 'hsa-mir-29a', 'hsa-mir-181d',
       'hsa-mir-155', 'hsa-mir-181c', 'hsa-let-7d', 'hsa-mir-34c',
       'hsa-let-7b', 'hsa-mir-15a', 'hsa-mir-134', 'hsa-mir-98', 'hsa-mir-143',
       'hsa-mir-135b', 'BRCA1', 'SNAI2', 'VIM', 'CD44', 'CDH1', 'FGFR1', 'PGR',
       'ESR1', 'GATA3', 'CCND1', 'KRT18', 'EPCAM', 'PIK3CA', 'TWIST1', 'SNAI1',
       'KRT17', 'FOXA1', 'MYC', 'BRCA2', 'TP53', 'ERBB2', 'AKT1', 'EGFR_rna',
       'ZEB1', 'MKI67', 'TGFBR2', 'CLDN3', 'AR', 'ZEB2', 'KRT19', 'BCL2_rna',
       'PTEN_rna', 'CLDN7', 'MUC1', 'KRT5', 'KRT14', 'CLDN4', 'CD24'],
      dtype='object')

In [64]:
print(df_geny)

              ERALPHA        PR      HER2  EGFR_mirnaprot      P53  \
TCGA-WT-AB41  3.85540 -0.219920 -0.093352        -0.57223 -0.19534   
TCGA-BH-A0H5  2.31530 -0.157000 -0.067206        -0.46202 -0.58071   
TCGA-C8-A27A  2.45540  0.071407  0.002673        -0.46112  0.36894   
TCGA-AC-A23C  3.05480 -0.239620  2.002200        -0.22857 -1.15750   
TCGA-A8-A085  3.27370 -0.178650  0.135140        -0.55378 -0.28867   
...               ...       ...       ...             ...      ...   
TCGA-A2-A0CS  2.67140  0.582550 -0.056856        -0.64176 -0.15178   
TCGA-LL-A8F5 -1.09020 -0.171540 -0.346880         0.57231  0.54341   
TCGA-E9-A229  0.79379  0.264860 -0.618610        -0.18988 -0.23675   
TCGA-D8-A1Y0  2.87270  0.359090 -0.282480        -0.16115  0.23611   
TCGA-BH-A0DH  2.41810  0.243550  0.233270        -0.29751 -0.44530   

              PTEN_mirnaprot     MEK1  BCL2_mirnaprot       BAX  PARPCLEAVED  \
TCGA-WT-AB41        0.253260  0.34339         1.04230 -0.184940    -0.351010   

In [66]:
df_geny_reset = df_geny.reset_index().rename(columns={'index': 'ID.1'})

merged_df2 = df_geny_reset.merge(
    df_ogolne,
    how='inner',
    left_on='ID.1',
    right_on='cases.submitter_id'
)

print("Liczba wspólnych pacjentów:", merged_df2['cases.submitter_id'].nunique())


Liczba wspólnych pacjentów: 855


In [68]:
merged_df2.to_csv('merged_data.tsv', sep='\t', index=False)
